In [5]:
import sys; sys.path.append('..')
import random, math
import pyzx as zx
from pyzx.linalg import *
from fractions import Fraction
%config InlineBackend.figure_format = 'svg'

In [11]:
import time

qubit_amount = 12
iterations = 100

print("Qubit count:", qubit_amount)

times = []

for depth in [200,400,800,1600, 2500]:
    count = 0
    t = time.time()
    for i in range(1,iterations+1):
        if i%50 == 0:
            print(i, end=' ')
        seed = random.randint(0,100000)
        random.seed(seed)
        circ = zx.cliffordT(qubit_amount, depth,p_t=0.2)
        g = circ.copy()
        zx.clifford_simp(g,quiet=True)
        g.normalise()
        #c = continuous_extract(g)
        zx.extract.circuit_extract(g)
        c = zx.Circuit.from_graph(g)
        count += len(c.gates)
        #if not zx.compare_tensors(circ.to_tensor(),c.to_tensor()):
        #    raise Exception("Not equal")
    times.append((time.time()-t)/depth)
    print("\nStarting depth: ", depth)
    print("Average gate count:", count/float(iterations))
    print("Reduction: {:.1%}\n".format(1-(count/float(iterations))/depth))

Qubit count: 12
50 100 
Starting depth:  200
Average gate count: 412.0
Reduction: -106.0%

50 100 
Starting depth:  400
Average gate count: 815.79
Reduction: -103.9%

50 100 
Starting depth:  800
Average gate count: 1572.1
Reduction: -96.5%



KeyboardInterrupt: 

In [130]:
times

[0.026000468730926513,
 0.053905845284461976,
 0.13439435362815857,
 0.2084313890337944,
 0.25764179964065553]

Experiments:


Qubit count: 4


50 100 
Starting depth:  200
Average gate count: 105.44
Reduction: 47.3%

50 100 
Starting depth:  400
Average gate count: 200.95
Reduction: 49.8%

50 100 
Starting depth:  800
Average gate count: 387.89
Reduction: 51.5%

50 100 
Starting depth:  1600
Average gate count: 757.31
Reduction: 52.7%

50 100 
Starting depth:  2500
Average gate count: 1178.89
Reduction: 52.8%

50 100 
Starting depth:  5000
Average gate count: 2339.43
Reduction: 53.2%

times = `[0.014082347154617309,
 0.015004873871803284,
 0.014446367621421813,
 0.01545741394162178,
 0.016793507194519042,
 0.01939831700325012]`


Qubit count: 7


50 100 
Starting depth:  200
Average gate count: 152.22
Reduction: 23.9%

50 100 
Starting depth:  400
Average gate count: 276.4
Reduction: 30.9%

50 100 
Starting depth:  800
Average gate count: 526.31
Reduction: 34.2%

50 100 
Starting depth:  1600
Average gate count: 1017.35
Reduction: 36.4%

50 100 
Starting depth:  2500
Average gate count: 1581.45
Reduction: 36.7%


Qubit count: 10


50 100 
Starting depth:  200
Average gate count: 191.46
Reduction: 4.3%

50 100 
Starting depth:  400
Average gate count: 343.82
Reduction: 14.0%

50 100 
Starting depth:  800
Average gate count: 650.12
Reduction: 18.7%

50 100 
Starting depth:  1600
Average gate count: 1259.9
Reduction: 21.3%

50 100 
Starting depth:  2500
Average gate count: 1962.36
Reduction: 21.5%

50 100 
Starting depth:  5000
Average gate count: 3851.59
Reduction: 23.0%

times = `[0.01880471348762512,
 0.020604894757270814,
 0.022415057718753816,
 0.024065009653568267,
 0.025690890407562256,
 0.028920268106460572]`
 

Qubit count: 12


50 100 
Starting depth:  200
Average gate count: 215.54
Reduction: -7.8%

50 100 
Starting depth:  400
Average gate count: 390.13
Reduction: 2.5%

50 100 
Starting depth:  800
Average gate count: 721.57
Reduction: 9.8%

50 100 
Starting depth:  1600
Average gate count: 1415.25
Reduction: 11.5%

50 100 
Starting depth:  2500
Average gate count: 2176.43
Reduction: 12.9%


Qubit count: 15


50 100 
Starting depth:  200
Average gate count: 249.72
Reduction: -24.9%

50 100 
Starting depth:  400
Average gate count: 451.7
Reduction: -12.9%

50 100 
Starting depth:  800
Average gate count: 851.36
Reduction: -6.4%

50 100 
Starting depth:  1600
Average gate count: 1646.96
Reduction: -2.9%

50 100 
Starting depth:  2500
Average gate count: 2526.7
Reduction: -1.1%


Qubit count: 20



50 100 
Starting depth:  200
Average gate count: 295.62
Reduction: -47.8%

50 100 
Starting depth:  400
Average gate count: 546.46
Reduction: -36.6%

50 100 
Starting depth:  800
Average gate count: 1027.37
Reduction: -28.4%

50 100 
Starting depth:  1600
Average gate count: 1997.68
Reduction: -24.9%

50 100 
Starting depth:  2500
Average gate count: 3084.87
Reduction: -23.4%

times = `[0.026000468730926513,
 0.053905845284461976,
 0.13439435362815857,
 0.2084313890337944,
 0.25764179964065553]`

In [10]:
#nice circuit
#qubit_amount = 4
#depth = 50
#random.seed(1337)

qubit_amount = 40
depth = 1200
#random.seed(1340)

#circ = zx.cliffordT(qubit_amount, depth,p_t=0.2)
circ = zx.generate.cnots(qubit_amount,depth)
#zx.draw(circ)#,h_edge_draw='box')
g = circ.copy()
#zx.simplify.phase_free_simp(g,quiet=False)
zx.clifford_simp(g,quiet=False)
g.normalise()
gg = g
g = gg.copy()
#zx.draw(g,labels=True)

spider_simp: 403. 286. 204. 140. 78. 47. 28. 2.  8 iterations
pivot_simp: 237. 109. 60. 39. 24. 15. 15. 9. 10. 9. 7. 7. 8. 7. 6. 8. 7. 6. 7. 5. 6. 5. 6. 5. 7. 5. 5. 5. 6. 5. 5. 5. 5. 4. 4. 5. 5. 4. 4. 3. 5. 4. 2. 1.  44 iterations


In [379]:
for block in range(2,qubit_amount+1):
    g = gg.copy()
    zx.extract.circuit_extract(g,cnot_blocksize=block)
    zx.simplify.to_rg(g)
    zx.simplify.to_rg(g)
    c = zx.Circuit.from_graph(g)
    #c = continuous_extract(g)
    print(block, len(c.gates))

2 731
3 622
4 596
5 582
6 591
7 587
8 579
9 591
10 597
11 594
12 608
13 596
14 645
15 634
16 635
17 635
18 635
19 635
20 635
21 635
22 634
23 634
24 629
25 630
26 640
27 634
28 646
29 657
30 635


In [11]:
from pyzx.circuit import SWAP
g = gg.copy()
c = zx.extract.streaming_extract(g)
# swaps = []
# for g in c.gates.copy():
#     if isinstance(g, SWAP):
#         swaps.append(g)
#         c.gates.remove(g)
print(len(c.gates))
g = c.to_graph(compress_rows=True)
#zx.extract.circuit_extract(g)
zx.simplify.id_simp(g,quiet=True)
zx.simplify.to_gh(g)
#zx.simplify.to_rg(g)
#zx.simplify.to_rg(g)
c = zx.Circuit.from_graph(g,split_phases=False)
import pyzx.phasepoly
c2, par = pyzx.phasepoly.circuit_phase_polynomial_blocks(c, optimize=True)
print(len(c.gates))
print(len(c2.gates))
#zx.draw(c2.to_graph(),figsize=(10,3),rows=(0,150))

MemoryError: 